# Program 1                DATA AGGREGATION

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os as os
import datetime

Read in dataframes from gzip files

In [3]:
def gz_to_df(name):
    df=pd.DataFrame()
    df = pd.read_csv(name,header=None,usecols=[1,3,11],parse_dates=[1])
    cols = ["DATE","STORENUMBER","SALES"]
    df.columns=cols
    df = df.loc[:,['STORENUMBER','DATE','SALES']]
    df.sort_values(by=['STORENUMBER','DATE'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df

Aggregate transactions by day

In [4]:
def aggregate_transactions_by_day(data):
    data.sort_values(by=['DATE'],inplace=True)
    #set datetime as index
    data=data.set_index('DATE')
    #resample and sum by day
    data=data.resample('D').sum()
    #restore date columns
    data=data.reset_index()
#    print('Aggregated One Dataframe')
    return data

Aggregate one dataframe at a time

In [5]:
def aggregate_one_dataframe(df):  
    df_agg=pd.DataFrame()
    stores = df['STORENUMBER'].unique()
    for store in stores:
        dfs = pd.DataFrame()
        dfs = df[df['STORENUMBER'] == store]
        dfs = dfs.drop(['STORENUMBER'],axis=1)
        dfs = aggregate_transactions_by_day(dfs)
        days = dfs.shape[0]
        storenumber = [store for i in range(days)]
        dfs['STORENUMBER'] = storenumber
        df_agg = pd.concat([df_agg,dfs],axis=0,ignore_index=True)
    df_agg = df_agg.loc[:,['STORENUMBER','DATE','SALES']]
    df_agg.reset_index(drop=True,inplace=True)
    return df_agg

Read, aggregate and concatenate all dataframes

In [6]:
def process_all_data_files():
    data = pd.DataFrame();df=pd.DataFrame()
    fnames=os.listdir(os.getcwd()+'/SF_data')
    for name in fnames:
        name=os.getcwd() + '/SF_data/' + str(name)
        if name.endswith('.gz'):
            df = gz_to_df(name)
            df = aggregate_one_dataframe(df)
            print('Aggregated One DataFrame')
            data = pd.concat([data,df],axis=0,ignore_index=True)
        else: continue
    data.to_feather('SF_data/SF_daily_data.feather')
    print('Data Stored To agg_data/daily_data.feather')
    return

In [7]:
process_all_data_files()

Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Aggregated One DataFrame
Data Stored To SF_data/SF_daily_data.feather


#### CLOSE AND HALT NOTEBOOK WHEN COMPLETED
After execution of this notebook it should be 'close and halt" under the File menu selection. This notebook perform so many caluculations tht it tends to corrupt memory locations.  Since the majority of the operations are in dataframe aggregation and mergers or in disk memory access multi-processing or GPU operations will not improve speed.  Distributed processing under DASK would work but required a cpu/gpu network and this is intended to operate on a single host for demonstration purposes.  'Close and halt' will start a cleaner.

### Proceed to Program 2:  SFFeatures.ipynb